In [1]:
import torch
from torch import nn 
from torch.nn import functional as F



In [6]:
X = torch.rand((2, 20))
X

tensor([[0.7714, 0.6921, 0.1398, 0.2806, 0.2174, 0.2120, 0.6952, 0.0275, 0.1852,
         0.5088, 0.7565, 0.1195, 0.7433, 0.5003, 0.6993, 0.0034, 0.4097, 0.9332,
         0.9515, 0.0555],
        [0.4160, 0.1287, 0.3307, 0.7352, 0.2086, 0.6068, 0.8160, 0.3115, 0.9350,
         0.4192, 0.8132, 0.9023, 0.2676, 0.1212, 0.8606, 0.6850, 0.6898, 0.1776,
         0.7385, 0.7751]])

In [8]:
net = nn.Sequential(
    nn.Linear(20, 256),
    nn.ReLU(),
    nn.Linear(256, 1)
)

net(X)

tensor([[-0.0632],
        [ 0.0023]], grad_fn=<AddmmBackward0>)

In [13]:
class MLP(nn.Module):
    # 用模型参数声明层。这里，我们声明两个全连接的层
    def __init__(self):
        # 调用MLP的父类Module的构造函数来执行必要的初始化。这样，在类实例化时也可以指定其他函数参数，例如模型参数params（稍后将介绍）
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)

    # 定义模型的前向传播，即根据输入X返回所需的模型输出
    def forward(self, X):
        # 注意，这里我们使用ReLU的函数版本，其在nn.functional模块中定义。
        return self.out(F.relu(self.hidden(X)))


In [14]:
net = MLP()
net(X)

tensor([[ 0.0556,  0.1683, -0.0958,  0.0797,  0.1769, -0.0504,  0.0167, -0.0133,
          0.2007, -0.1678],
        [-0.0478,  0.1134, -0.1759,  0.0988,  0.0079, -0.0134, -0.2283, -0.2379,
          0.1407, -0.2447]], grad_fn=<AddmmBackward0>)

In [15]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            # 这里，module是Module子类的一个实例。我们把它保存在'Module'类的成员变量_modules中。_module的类型是OrderedDict
            self._modules[str(idx)] = module
    
    def forward(self, X):
        # OrderedDict保证了按照成员添加的顺序遍历它们
        for block in self._modules.values():
            X = block(X)
        return X

In [16]:
net = MySequential(
    nn.Linear(20, 256),
    nn.ReLU(),
    nn.Linear(256, 10)
)
net(X)

tensor([[-0.1358, -0.1127,  0.2307, -0.0236, -0.2704, -0.0307,  0.1185,  0.2977,
         -0.0987, -0.0058],
        [-0.0495, -0.0240,  0.1192,  0.1559, -0.3757, -0.1369,  0.1451,  0.1748,
         -0.2489,  0.0073]], grad_fn=<AddmmBackward0>)

In [25]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # 不计算梯度的随机权重参数。因此其在训练期间保持不变
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight)+1)
        X = self.linear(X)

        while X.abs().sum() > 1: # 求X的L1范数
            X /= 2
        return X.sum()

In [26]:
net = FixedHiddenMLP()
net(X)

tensor(0.0444, grad_fn=<SumBackward0>)

In [27]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64),
                                 nn.ReLU(),
                                 nn.Linear(64, 32),
                                 nn.ReLU())
        self.linear = nn.Linear(32, 16)
    
    def forward(self, X):
        return self.linear(self.net(X))
    
chimera = nn.Sequential(NestMLP(),
                        nn.Linear(16, 20),
                        FixedHiddenMLP())
chimera(X)

tensor(-0.1983, grad_fn=<SumBackward0>)